# **Penting**
- Jangan mengubah atau menambahkan cell text yang sudah disediakan, Anda hanya perlu mengerjakan cell code yang sudah disediakan.
- Pastikan seluruh kriteria memiliki output yang sesuai, karena jika tidak ada output dianggap tidak selesai.
- Misal, Anda menggunakan df = df.dropna() silakan gunakan df.isnull().sum() sebagai tanda sudah berhasil. Silakan sesuaikan seluruh output dengan perintah yang sudah disediakan.
- Pastikan Anda melakukan Run All sebelum mengirimkan submission untuk memastikan seluruh cell berjalan dengan baik.
- Pastikan Anda menggunakan variabel df dari awal sampai akhir dan tidak diperbolehkan mengganti nama variabel tersebut.
- Hapus simbol pagar (#) pada kode yang bertipe komentar jika Anda menerapkan kriteria tambahan
- Biarkan simbol pagar (#) jika Anda tidak menerapkan kriteria tambahan
- Pastikan Anda mengerjakan sesuai section yang sudah diberikan tanpa mengubah judul atau header yang disediakan.

# **1. Import Library**
Pada tahap ini, Anda perlu mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning.

In [17]:
#Type your code here
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import joblib


# **2. Memuat Dataset dari Hasil Clustering**
Memuat dataset hasil clustering dari file CSV ke dalam variabel DataFrame.

In [18]:
# Gunakan dataset hasil clustering yang memiliki fitur Target
# Silakan gunakan dataset data_clustering jika tidak menerapkan Interpretasi Hasil Clustering [Advanced]
# Silakan gunakan dataset data_clustering_inverse jika menerapkan Interpretasi Hasil Clustering [Advanced]
# Lengkapi kode berikut
df = pd.read_csv("/content/data_clustering_inverse.csv")

In [19]:
# Tampilkan 5 baris pertama dengan function head.
df.head(5)

,TransactionAmount,TransactionType,Location,Channel,CustomerAge,CustomerOccupation,TransactionDuration,LoginAttempts,AccountBalance,AgeGroup,TransactionSize,Target
0,14.09,Debit,San Diego,ATM,70.0,Doctor,81.0,1.0,5112.21,Lansia,Sedikit,0
1,376.24,Debit,Houston,ATM,68.0,Doctor,141.0,1.0,13758.91,Lansia,Sedang,3
2,126.29,Debit,Mesa,Online,19.0,Student,56.0,1.0,1122.35,Remaja,Sedikit,2
3,184.50,Debit,Raleigh,Online,26.0,Student,25.0,1.0,8569.06,Dewasa Muda,Sedikit,0
4,92.15,Debit,Oklahoma City,ATM,18.0,Student,172.0,1.0,781.68,Remaja,Sedikit,2


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2182 entries, 0 to 2181
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   TransactionAmount    2182 non-null   float64
 1   TransactionType      2152 non-null   object 
 2   Location             2154 non-null   object 
 3   Channel              2159 non-null   object 
 4   CustomerAge          2182 non-null   float64
 5   CustomerOccupation   2162 non-null   object 
 6   TransactionDuration  2182 non-null   float64
 7   LoginAttempts        2182 non-null   float64
 8   AccountBalance       2182 non-null   float64
 9   AgeGroup             2182 non-null   object 
 10  TransactionSize      2182 non-null   object 
 11  Target               2182 non-null   int64  
dtypes: float64(5), int64(1), object(6)
memory usage: 204.7+ KB


# **3. Data Splitting**
Tahap Data Splitting bertujuan untuk memisahkan dataset menjadi dua bagian: data latih (training set) dan data uji (test set).

In [21]:
# Menggunakan train_test_split() untuk melakukan pembagian dataset.

df.dropna(inplace=True)
# Pisahkan fitur dan target
X = df.drop('Target', axis=1)
y = df['Target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# --- Label Encoding kategorikal ---
categorical_columns = X_train.select_dtypes(include=['object']).columns
label_encoders = {}

for col in categorical_columns:
    le = LabelEncoder()
    X_train[col] = le.fit_transform(X_train[col])
    X_test[col] = le.transform(X_test[col])
    label_encoders[col] = le

# --- Normalisasi  numerik ---
numeric_columns = X_train.select_dtypes(include=['float64']).columns
scaler = StandardScaler()

# Fit hanya di train, transform di test
X_train[numeric_columns] = scaler.fit_transform(X_train[numeric_columns])
X_test[numeric_columns] = scaler.transform(X_test[numeric_columns])
X_test.head()

,TransactionAmount,TransactionType,Location,Channel,CustomerAge,CustomerOccupation,TransactionDuration,LoginAttempts,AccountBalance,AgeGroup,TransactionSize
1653,-0.132824,1,15,2,1.498053,2,-0.819538,0.0,-0.170964,2,1
873,-1.109688,1,10,1,-0.988210,3,-1.146191,0.0,-1.124867,1,2
1415,0.424284,1,3,2,-1.157728,3,0.401858,0.0,-1.039422,3,1
1035,-0.887269,1,3,2,0.537451,0,1.978312,0.0,0.740245,0,2
1921,1.334569,1,24,1,-0.988210,1,-1.103584,0.0,-0.054206,1,1


# **4. Membangun Model Klasifikasi**
Setelah memilih algoritma klasifikasi yang sesuai, langkah selanjutnya adalah melatih model menggunakan data latih.

Berikut adalah rekomendasi tahapannya.
1. Menggunakan algoritma klasifikasi yaitu Decision Tree.
2. Latih model menggunakan data yang sudah dipisah.

In [22]:
# Buatlah model klasifikasi menggunakan Decision Tree
model = DecisionTreeClassifier()
model.fit(X_train, y_train)

DecisionTreeClassifier()

In [23]:
# Menyimpan Model
import joblib
joblib.dump(model, 'decision_tree_model.h5')

['decision_tree_model.h5']

# **5. Memenuhi Kriteria Skilled dan Advanced dalam Membangun Model Klasifikasi**



**Biarkan kosong jika tidak menerapkan kriteria skilled atau advanced**

In [24]:
# Melatih model menggunakan algoritma klasifikasi scikit-learn selain Decision Tree.
knn = KNeighborsClassifier().fit(X_train, y_train)
nb = GaussianNB().fit(X_train, y_train)

In [25]:
# Menampilkan hasil evaluasi akurasi, presisi, recall, dan F1-Score pada seluruh algoritma yang sudah dibuat.
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    results = {
        'Accuracy': accuracy_score(y_test, y_pred),
        'Precision': precision_score(y_test, y_pred, average='weighted'),  # atau 'macro'
        'Recall': recall_score(y_test, y_pred, average='weighted'),
        'F1-Score': f1_score(y_test, y_pred, average='weighted')
    }
    return results

# Mengevaluasi setiap model dan mengumpulkan hasilnya
results = {
    'K-Nearest Neighbors (KNN)': evaluate_model(knn, X_test, y_test),
    'Decision Tree (DT)': evaluate_model(model, X_test, y_test),
    'Naive Bayes (NB)': evaluate_model(nb, X_test, y_test)
}

# Buat DataFrame untuk meringkas hasil
summary_df = pd.DataFrame(columns=['Model', 'Accuracy', 'Precision', 'Recall', 'F1-Score'])

# Isi DataFrame dengan hasil
rows = []
for model_name, metrics in results.items():
    rows.append({
        'Model': model_name,
        'Accuracy': metrics['Accuracy'],
        'Precision': metrics['Precision'],
        'Recall': metrics['Recall'],
        'F1-Score': metrics['F1-Score']
    })

# Konversi ke DataFrame
summary_df = pd.DataFrame(rows)

# Tampilkan DataFrame
summary_df

,Model,Accuracy,Precision,Recall,F1-Score
0,K-Nearest Neighbors (KNN),0.973621,0.974460,0.973621,0.973776
1,Decision Tree (DT),1.000000,1.000000,1.000000,1.000000
2,Naive Bayes (NB),0.988010,0.988191,0.988010,0.988016


In [26]:
# Menyimpan Model Selain Decision Tree
# Model ini bisa lebih dari satu
import joblib
joblib.dump(knn, 'explore_knn_classification.h5')

['explore_knn_classification.h5']

Hyperparameter Tuning Model

Pilih salah satu algoritma yang ingin Anda tuning

In [27]:
# Lakukan Hyperparameter Tuning dan Latih ulang.
# Lakukan dalam satu cell ini saja.
from sklearn.model_selection import RandomizedSearchCV
from sklearn.neighbors import KNeighborsClassifier
import numpy as np


# Definisikan ruang pencarian untuk Random Search
param_dist = {
    'n_neighbors': np.arange(3, 31, 2),
    'weights': ['uniform', 'distance'],
    'metric': ['minkowski', 'manhattan', 'euclidean'],
    'p': [1, 2]
}

# Inisialisasi RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=knn,
    param_distributions=param_dist,
    n_iter=20,
    cv=5,
    n_jobs=-1,
    verbose=2,
    random_state=42
)

# Latih RandomizedSearchCV
random_search.fit(X_train, y_train)

# Output hasil terbaik
print(f"Best parameters (Random Search KNN): {random_search.best_params_}")

# Simpan model terbaik
best_knn_random = random_search.best_estimator_


Fitting 5 folds for each of 20 candidates, totalling 100 fits
Best parameters (Random Search KNN): {'weights': 'distance', 'p': 1, 'n_neighbors': np.int64(27), 'metric': 'manhattan'}


In [28]:
# Menampilkan hasil evaluasi akurasi, presisi, recall, dan F1-Score pada algoritma yang sudah dituning.
y_pred_random = best_knn_random.predict(X_test)
results = {
        'Accuracy': accuracy_score(y_test, y_pred_random),
        'Precision': precision_score(y_test, y_pred_random, average='weighted'),  # atau 'macro'
        'Recall': recall_score(y_test, y_pred_random, average='weighted'),
        'F1-Score': f1_score(y_test, y_pred_random, average='weighted')
    }
results

{'Accuracy': 0.9904076738609112,
 'Precision': 0.9905150651515262,
 'Recall': 0.9904076738609112,
 'F1-Score': 0.9904139986854095}

In [29]:
# Menyimpan Model hasil tuning
import joblib
joblib.dump(best_knn_random, 'tuning_classification.h5')

['tuning_classification.h5']

End of Code